In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanpy.external as scex
import sklearn.metrics
from matplotlib import pyplot as plt
import matplotlib.cm
import bbknn
import seaborn as sns
from scipy import stats
from sklearn.mixture import GaussianMixture as GMM
from scipy.stats import norm

In [ ]:
#adata.uns['log1p']['base'] = None

In [ ]:
%matplotlib inline

In [ ]:
inputFile = "../../data/sarcoma/sarcPDX_Treat_UPS2236/wk1_filtered_feature_bc_matrix.h5"

In [ ]:
writeLoc = "../../write/"

fileName = "sarcPDX_Treat_ctw1_UPS2236_V5"

resultsFile = writeLoc + fileName + '.h5ad'  # the file that will store the analysis results
resultsFileQC = writeLoc + fileName + '_QC.h5ad'  
#results_file_ = writeLoc + fileName + '_fTry.h5ad'

In [ ]:
adata =  sc.read_10x_h5(inputFile, gex_only=False)
adata


In [ ]:
np.unique(adata.var.genome)

In [ ]:
#split up input into genes and hashes 
hto = adata[:,adata.var["feature_types"] == "Antibody Capture"]
adata = adata[:,adata.var["feature_types"] == "Gene Expression"]
adata.obs = pd.DataFrame(hto.X.todense(), columns=hto.var_names, index=adata.obs.index)

In [ ]:
adata.var_names_make_unique()

In [ ]:
hto.var_names

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
adata

In [ ]:
valX = adata.X>0
minGenes=450
minCells=3

fig, axs = plt.subplots(2, figsize=(8, 4))

numCellwExp = valX.sum(axis=1)
axs[0].hist(np.log(numCellwExp), bins=100)#, log=True)
axs[0].axvline(np.log(minGenes), color='k', linestyle='dashed', linewidth=1)
axs[0].set_xlabel('sum cell with exp counts')
axs[0].set_ylabel('counts')

numGeneswExp = valX.sum(axis=0).T
axs[1].hist(np.log(numGeneswExp+1), bins=100, log=True)
axs[1].axvline(np.log(minCells+1), color='k', linestyle='dashed', linewidth=1)
axs[1].set_xlabel('sum cell have exp counts')
axs[1].set_ylabel('counts')

fig.show()

In [ ]:
sc.pp.filter_cells(adata, min_genes = minGenes)
sc.pp.filter_genes(adata, min_cells = minCells)

In [ ]:
from collections import Counter
print(Counter(adata.var_names.str.startswith('GRCh38_')))
print(Counter(adata.var_names.str.startswith('GRCh38_MT-')))

print(Counter(adata.var_names.str.startswith('mm10___')))
print(Counter(adata.var_names.str.startswith('mm10___mt-')))

In [ ]:
adata.var['human'] = adata.var_names.str.startswith('GRCh38_')  # annotate the group of human genes as 'human'
adata.var['mouse'] = adata.var_names.str.startswith('mm10___')  # annotate the group of mouse genes as 'mouse'

sc.pp.calculate_qc_metrics(adata, qc_vars=['human'], percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(adata, qc_vars=['mouse'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['pct_counts_mouse', 'pct_counts_human'],
             jitter=0.4, multi_panel=True, log = True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'],
             jitter=0.4, multi_panel=True)

In [ ]:
sum(adata.obs.pct_counts_mouse < 1)

In [ ]:
adata

In [ ]:
adata = adata[adata.obs.pct_counts_mouse < 1, :]
adata = adata[:,adata.var_names.str.startswith('GRCh38_')]
adata

In [ ]:
sc.pl.violin(adata, ['pct_counts_mouse', 'pct_counts_human'],
             jitter=0.4, multi_panel=True, log = True)

In [ ]:
adata.var['hMT'] = adata.var_names.str.startswith('GRCh38_MT-')  # annotate the group of human mitochondrial genes as 'hMT'
#adata.var['hMT'] = adata.var_names.str.startswith('mm10___mt-')  # annotate the group of mouse mitochondrial genes as 'mmt'

sc.pp.calculate_qc_metrics(adata, qc_vars=['hMT'], percent_top=None, log1p=False, inplace=True)
#sc.pp.calculate_qc_metrics(adata, qc_vars=['mmt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"], #pct_counts_mmt
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_hMT')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 5250, :]
adata = adata[adata.obs.n_genes_by_counts > 1000, :]
adata = adata[adata.obs.total_counts < 20000, :]
adata = adata[adata.obs.pct_counts_hMT < 10, :]

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"], #pct_counts_mmt
             jitter=0.4, multi_panel=True)

In [ ]:
hto.var_names

In [ ]:
import scanpy.external as sce
sce.pp.hashsolo(adata, hto.var_names)#number_of_noise_barcodes = 1
adata.obs.head()

In [ ]:
sc.pl.heatmap(adata, hto.var_names, groupby="Classification", log=True)

In [ ]:
hto.var_names

In [ ]:
len(hto.var_names)

In [ ]:
Counter(adata.obs["Classification"])

In [ ]:
hashCounts = adata.obs[hto.var_names]
hashCounts.describe([0.01,.1,.2,.3,.4,.5,.6,.7,.8,.9,0.99])

In [ ]:
#hashCount = np.log1p(adata.obs[hashName])
#len(hashCount)

In [ ]:
fig, axs = plt.subplots(2,5)
dfHashBoundry = pd.DataFrame(np.zeros(len(hto.var_names)),hto.var_names, columns=["boundry"])
gmm = GMM(n_components = 2, random_state=10,  n_init=5, covariance_type = 'full', means_init=[[2],[7]])#
#binEx = np.arange(0.5,10,10/200).reshape(-1,1)

for i, hashName in enumerate(hto.var_names):
    hashCount = np.log1p(adata.obs[hashName]).values
    trimV = 1*len(hashCount)//100
    hashCount = np.sort(hashCount)[trimV:].reshape(-1, 1)
    fitGMM = gmm.fit(hashCount)
    mean = fitGMM.means_  
    covs  = fitGMM.covariances_
    weights = fitGMM.weights_
    #print(mean)
    binEx = np.arange(min(mean),max(mean),0.1).reshape(-1,1)
    fitGmmBound = fitGMM.predict(binEx)
    #print(fitGmmBound)
    hashBoundry = binEx[np.where(fitGmmBound == 1)[0][0]][0]
    
    dfHashBoundry.loc[hashName] = hashBoundry
    
    x_axis = np.arange(0, 12, 0.1)
    y_axis0 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
    y_axis1 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[1][0][0])))*weights[1] # 2nd gaussian

    # Plot 2
    axs[i//5,i%5].set_title(hashName)
    #axs[i//5,i%5].axvline(naiveBoundry, c='C3', linestyle='dashed', linewidth=1) #red
    axs[i//5,i%5].axvline(hashBoundry, c='C2', linestyle='dashed', linewidth=1)  #green
    axs[i//5,i%5].hist(hashCount, density=True, color='black', bins=100)        
    axs[i//5,i%5].plot(x_axis, y_axis0, lw=3, c='C6')                            #pink
    axs[i//5,i%5].plot(x_axis, y_axis1, lw=3, c='C1')                            #orange
    axs[i//5,i%5].plot(x_axis, y_axis0+y_axis1, lw=3, c='C0', ls=':')            #dotted blue
    
plt.tight_layout(pad=1.0)
plt.rcParams["figure.figsize"] = (20,5)
plt.show()

In [ ]:
hashIDs = hashCounts.copy()
hashID = hto.var_names
for hashName in hto.var_names:
    print(hashName)
    print(dfHashBoundry.loc[hashName].values[0])
    hashIDs.loc[:,hashName] = np.log1p(hashCounts.loc[:,hashName]) > dfHashBoundry.loc[hashName].values[0]
hashIDs

In [ ]:
classification = np.empty(len(adata), dtype="object")
i = 0
for cellBar, hashBool in hashIDs.iterrows():
    numHashes = sum(hashBool)
    if (numHashes == 1):
        classif = hashID[hashBool.values].values[0]
    elif (numHashes > 1):
        classif = "doublet"
    else:
        classif = "negative"
    classification[i] = classif
    i = i + 1
    #break

In [ ]:
adata.obs["ClassGMM"] = classification
adata.obs["ClassGMM"].value_counts()

In [ ]:
sc.pl.heatmap(adata, hto.var_names, groupby="ClassGMM", log=True)#, save = f"_{figName}_hash.png")

In [ ]:
sc.pl.heatmap(adata, hto.var_names, groupby="Classification", log=True)#, save = f"_{figName}_hash.png")

In [ ]:
singlets = [x in hto.var_names for x in adata.obs["Classification"]]
adata = adata[singlets,]
adata

In [ ]:
sarc =  {
    'UPS2236_1wk1_J':'wk1',  
    'UPS2236_Control1_G':'ctl',
    'UPS2236_1wk4_C':'wk1',
    'UPS2236_1wk2_A':'wk1',
    'UPS2236_1wk3_B':'wk1',
    'UPS2236_Control2_H':'ctl',
    'UPS2236_Control4_E':'ctl',
    'UPS2236_Control3_D':'ctl',
    'UPS2236_Control5_F':'ctl'}
adata.obs["sarc"] = [sarc[x] for x in adata.obs["Classification"]]

In [ ]:
resultsFileQC

In [ ]:
#['RPS8370_pSpCTRE_B0256', 'UPS2236_B0255']
#ups = [x in ["UPS"] for x in adata.obs["sarc"]]
#adata = adata[ups,]
#adata

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=12, format='svg')

In [ ]:
adata.var_names = [x.split("_")[1] for x in adata.var_names]
adata.var_names_make_unique()

In [ ]:
adata = adata[:,"MALAT1" != adata.var_names]

In [ ]:
#adata = sc.read(resultsFileQC)
adata.write(resultsFileQC)
adata.layers["counts"]=adata.X.copy()
adata

In [ ]:
#acta2 = adata[:,"ACTA2" == adata.var_names]
#print(sum(acta2.X.todense()>0))
#print(sum(acta2.X.todense()))
#acta2.var

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)#, exclude_highly_expressed=True

In [ ]:
#acta2 = adata[:,"ACTA2" == adata.var_names]
#print(sum(acta2.X.todense()>0))
#print(sum(acta2.X.todense()))
#acta2.var

In [ ]:
sc.pp.log1p(adata)

In [ ]:
#acta2 = adata[:,"ACTA2" == adata.var_names]
#print(sum(acta2.X.todense()>0))
#print(sum(acta2.X.todense()))
#acta2.var

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../../data/regev_lab_cell_cycle_genes_Human.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
adata_cc_genes = adata[:, cell_cycle_genes]
sc.tl.pca(adata_cc_genes)
sc.pl.pca(adata_cc_genes, color=['phase',"Classification"])

In [ ]:
sc.pp.highly_variable_genes(adata)#, n_top_genes=2000,  flavor="seurat_v3",layer="counts")
#min_disp=0.5,min_mean=0.0125, max_mean=3, span=0.3, n_bins=20,

In [ ]:
minMean = 0.045
maxMean = 2.9
minDisp = 0.35

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

means = adata.var[["means"]][adata.var[["means"]] > np.exp(-14)]#adata.var[["means"]],
axs[0].hist(np.log(means), bins=100)#, log=True),
axs[0].axvline(np.log(minMean), color='k', linestyle='dashed', linewidth=1)
axs[0].axvline(np.log(maxMean), color='k', linestyle='dashed', linewidth=1)
axs[0].set_title('Gene means counts')
axs[0].set_xlabel('means')
axs[0].set_ylabel('counts')

dispNorm = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-10)]#adata.var[["means"]],
axs[1].hist(np.log(dispNorm), bins=100)#, log=True),
axs[1].axvline(np.log(minDisp), color='k', linestyle='dashed', linewidth=1)
axs[1].set_title('Gene dispersions counts')
axs[1].set_xlabel('dispersions')
axs[1].set_ylabel('counts')

sc.pp.highly_variable_genes(adata, min_disp=minDisp, min_mean=minMean, max_mean=maxMean)
print(sum(adata.var.highly_variable))

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, np.logical_and(np.logical_not(adata.var.hMT), adata.var.highly_variable)]

In [ ]:
#sc.pp.regress_out(adata, ['S_score', 'G2M_score','total_counts', 'pct_counts_hMT'])

In [ ]:
#sc.pp.scale(adata, max_value=5)

In [ ]:
sc.tl.pca(adata, n_comps=100,svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=100, log=True)

In [ ]:
sc.pl.pca(adata, color=["phase"],ncols=3,annotate_var_explained=True, components=['1,2','1,3',"1,4", '2,3', '2,4', "3,4"])

In [ ]:
#adata.obsm["X_pca_1_40"] = adata.obsm["X_pca"][:,1:41]

In [ ]:
sc.pp.neighbors(adata, n_neighbors=25, n_pcs=60)#, use_rep="X_pca_1_40")

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution = 0.4)

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase",'sarc','Classification','total_counts', 'pct_counts_hMT'],ncols=2)

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='sarc')

In [ ]:
sc.pl.embedding_density(adata, basis='umap', key='umap_density_sarc')

In [ ]:
sc.pl.umap(adata, color=['COL1A1','DES','MDM2',"CDK4"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['FBN1','LUM',"POSTN"],ncols=3)

In [ ]:
sc.pl.umap(adata, color=["ACTA2","VIM"],ncols=3)

In [ ]:
sc.pl.umap(adata, color=['COL4A1','PECAM1','CD34',"RGS5"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['TPM2','TOP2A','VIM',"ISG15"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['KRT18','GAPDH','LDHB',"TPI1"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['PTPRC'], ncols=2, use_raw=True)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',use_raw=True)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_dotplot(adata, groupby = 'leiden',n_genes=4)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_dotplot(adata, groupby = 'leiden',n_genes=4)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
markGenes = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(1).values.tolist()[0]
markGenes

In [ ]:
sc.pl.umap(adata, color=markGenes,ncols=4)

In [ ]:
topHVG = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(100)
topHVG

In [ ]:
#topHVG.to_csv("hvg/treatedHvgups.csv", index=False)

compare 3 and 7 with mouse

compare 7 to other pdxs

In [ ]:
#sc.set_figure_params(scanpy=True, dpi=100, dpi_save=300, fontsize=18, format='svg',color_map="magma")
#sc.settings.figdir = "figures/scSarcTime_Grant/"

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_dotplot(adata, groupby = 'leiden',n_genes=4)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'Classification', method="wilcoxon", key_added="classMark")
pd.DataFrame(adata.uns["classMark"]['names']).head(10)

In [ ]:
sc.tl.rank_genes_groups(adata, groupby = 'sarc', method="wilcoxon", key_added="sarcMark")
pd.DataFrame(adata.uns["sarcMark"]['names']).head(10)

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_heatmap(adata, groupby = 'leiden')

In [ ]:
resultsFile

In [ ]:
adata.write(resultsFile)

In [ ]:
adata_ctwk1 = adata

In [ ]:
inputFile = "../../data/sarcoma/sarcPDX_Treat_UPS2236/wk3_filtered_feature_bc_matrix.h5"

In [ ]:
writeLoc = "../../write/"

fileName = "sarcPDX_Treat_wk3_UPS2236_V4"

resultsFile = writeLoc + fileName + '.h5ad'  # the file that will store the analysis results
resultsFileQC = writeLoc + fileName + '_QC.h5ad'  
#results_file_ = writeLoc + fileName + '_fTry.h5ad'

In [ ]:
adata =  sc.read_10x_h5(inputFile, gex_only=False)
adata

In [ ]:
np.unique(adata.var.genome)

In [ ]:
#split up input into genes and hashes 
hto = adata[:,adata.var["feature_types"] == "Antibody Capture"]
adata = adata[:,adata.var["feature_types"] == "Gene Expression"]
adata.obs = pd.DataFrame(hto.X.todense(), columns=hto.var_names, index=adata.obs.index)

In [ ]:
adata.var_names_make_unique()

In [ ]:
hto.var_names

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
adata

In [ ]:
valX = adata.X>0
minGenes=250
minCells=5

fig, axs = plt.subplots(2, figsize=(8, 8))

numCellwExp = valX.sum(axis=1)
axs[0].hist(np.log(numCellwExp), bins=100)#, log=True)
axs[0].axvline(np.log(minGenes), color='k', linestyle='dashed', linewidth=1)
axs[0].set_xlabel('sum cell with exp counts')
axs[0].set_ylabel('counts')

numGeneswExp = valX.sum(axis=0).T
axs[1].hist(np.log(numGeneswExp+1), bins=100, log=True)
axs[1].axvline(np.log(minCells+1), color='k', linestyle='dashed', linewidth=1)
axs[1].set_xlabel('sum cell have exp counts')
axs[1].set_ylabel('counts')

fig.show()

In [ ]:
sc.pp.filter_cells(adata, min_genes = minGenes)
sc.pp.filter_genes(adata, min_cells = minCells)

In [ ]:
adata

In [ ]:
np.log(700)

In [ ]:
valX = adata.X>0
numCellwExp = valX.sum(axis=1)
adata.obs["Keep"] = ["gra" if i else "les" for i in numCellwExp>700]

In [ ]:
from collections import Counter
print(Counter(adata.var_names.str.startswith('GRCh38_')))
print(Counter(adata.var_names.str.startswith('GRCh38_MT-')))

print(Counter(adata.var_names.str.startswith('mm10___')))
print(Counter(adata.var_names.str.startswith('mm10___mt-')))

In [ ]:
adata.var['human'] = adata.var_names.str.startswith('GRCh38_')  # annotate the group of human genes as 'human'
adata.var['mouse'] = adata.var_names.str.startswith('mm10___')  # annotate the group of mouse mitochondrial genes as 'mt'

sc.pp.calculate_qc_metrics(adata, qc_vars=['human'], percent_top=None, log1p=False, inplace=True)
sc.pp.calculate_qc_metrics(adata, qc_vars=['mouse'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['pct_counts_mouse', 'pct_counts_human'], groupby="Keep", jitter=0.1, multi_panel=True, log=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], groupby="Keep", jitter=0.1, multi_panel=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts'], jitter=0.4, multi_panel=True)

In [ ]:
sum(adata.obs.pct_counts_mouse < 2)

In [ ]:
adata = adata[adata.obs.pct_counts_mouse < 2, :]
adata = adata[:,adata.var_names.str.startswith('GRCh38_')]
adata

In [ ]:
sc.pl.violin(adata, ['pct_counts_mouse', 'pct_counts_human'],
             jitter=0.4, multi_panel=True, log = True)

In [ ]:
adata.var['hMT'] = adata.var_names.str.startswith('GRCh38_MT-')  # annotate the group of human mitochondrial genes as 'hMT'
#adata.var['hMT'] = adata.var_names.str.startswith('mm10___mt-')  # annotate the group of mouse mitochondrial genes as 'mmt'

sc.pp.calculate_qc_metrics(adata, qc_vars=['hMT'], percent_top=None, log1p=False, inplace=True)
#sc.pp.calculate_qc_metrics(adata, qc_vars=['mmt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"], #pct_counts_mmt
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"], #pct_counts_mmt
             jitter=0.4, multi_panel=True, groupby="Keep")

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_hMT')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

In [ ]:
adata

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 5500, :]
adata = adata[adata.obs.n_genes_by_counts > 1000, :]
adata = adata[adata.obs.total_counts < 25000, :]
adata = adata[adata.obs.pct_counts_hMT < 10, :]

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"], #pct_counts_mmt
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"], #pct_counts_mmt
             jitter=0.4, multi_panel=True, groupby="Keep")

In [ ]:
adata.obs

In [ ]:
hto.var_names

In [ ]:
import scanpy.external as sce
sce.pp.hashsolo(adata, hto.var_names)#number_of_noise_barcodes = 1
adata.obs.head()

In [ ]:
sc.pl.heatmap(adata, hto.var_names, groupby="Classification", log=True)

In [ ]:
hto.var_names

In [ ]:
len(hto.var_names)

In [ ]:
Counter(adata.obs["Classification"])

In [ ]:
singlets = [x in hto.var_names for x in adata.obs["Classification"]]
adata = adata[singlets,]
adata

In [ ]:
sarc =  {
         'UPS2236_4B_3_wk_chemo': "wk3",
         'UPS2236_2B_3_wk_chemo': "wk3",
         'UPS2236_1A_3_wk_chemo': "wk3",
         'UPS2236_2A_3_wk_chemo': "wk3",
         'UPS2236_1B_3_wk_chemo': "wk3",
         'UPS2236_3B_3_wk_chemo': "wk3",
         'UPS2236_3A_3_wk_chemo': "wk3",
         'UPS2236_4A_3_wk_chemo': "wk3"}
adata.obs["sarc"] = [sarc[x] for x in adata.obs["Classification"]]

In [ ]:
sarc =  {
         'UPS2236_4B_3_wk_chemo': "D",
         'UPS2236_2B_3_wk_chemo': "B",
         'UPS2236_1A_3_wk_chemo': "A",
         'UPS2236_2A_3_wk_chemo': "B",
         'UPS2236_1B_3_wk_chemo': "A",
         'UPS2236_3B_3_wk_chemo': "C",
         'UPS2236_3A_3_wk_chemo': "C",
         'UPS2236_4A_3_wk_chemo': "D"}
adata.obs["sample"] = [sarc[x] for x in adata.obs["Classification"]]

In [ ]:
adata.var_names = [x.split("_")[1] for x in adata.var_names]
adata.var_names_make_unique()

In [ ]:
adata = adata[:,"MALAT1" != adata.var_names]

In [ ]:
resultsFileQC

In [ ]:
#adata = sc.read(resultsFileQC)
adata.write(resultsFileQC)
adata.layers["counts"]=adata.X.copy()
adata

In [ ]:
resultsFileQC

In [ ]:
#['RPS8370_pSpCTRE_B0256', 'UPS2236_B0255']
#ups = [x in ["UPS"] for x in adata.obs["sarc"]]
#adata = adata[ups,]
#adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)#, exclude_highly_expressed=True

In [ ]:
sc.pp.log1p(adata)

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../../data/regev_lab_cell_cycle_genes_Human.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
adata_cc_genes = adata[:, cell_cycle_genes]
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color=['phase',"Classification"])

In [ ]:
sc.pp.highly_variable_genes(adata)#, flavor="seurat", n_top_genes=2000)
#min_disp=0.5,min_mean=0.0125, max_mean=3, span=0.3, n_bins=20,

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
minMean = 0.045
maxMean = 2.9
minDisp = 0.35

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

means = adata.var[["means"]][adata.var[["means"]] > np.exp(-14)]#adata.var[["means"]],
axs[0].hist(np.log(means), bins=100)#, log=True),
axs[0].axvline(np.log(minMean), color='k', linestyle='dashed', linewidth=1)
axs[0].axvline(np.log(maxMean), color='k', linestyle='dashed', linewidth=1)
axs[0].set_title('Gene means counts')
axs[0].set_xlabel('means')
axs[0].set_ylabel('counts')

dispNorm = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-10)]#adata.var[["means"]],
axs[1].hist(np.log(dispNorm), bins=100)#, log=True),
axs[1].axvline(np.log(minDisp), color='k', linestyle='dashed', linewidth=1)
axs[1].set_title('Gene dispersions counts')
axs[1].set_xlabel('dispersions')
axs[1].set_ylabel('counts')

sc.pp.highly_variable_genes(adata, min_disp=minDisp, min_mean=minMean, max_mean=maxMean)
print(sum(adata.var.highly_variable))

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, np.logical_and(np.logical_not(adata.var.hMT), adata.var.highly_variable)]

In [ ]:
sum(np.logical_and(np.logical_not(adata.var.hMT), adata.var.highly_variable))

In [ ]:
#sc.pp.regress_out(adata, ['S_score', 'G2M_score','total_counts', 'pct_counts_hMT'])

In [ ]:
#np.max(adata.X.todense())

In [ ]:
#sc.pp.scale(adata, max_value=4)

In [ ]:
sc.tl.pca(adata, n_comps=100,svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=100, log=True)

In [ ]:
sc.pl.pca(adata, color=["phase"],ncols=3,annotate_var_explained=True, components=['1,2','1,3',"1,4", '2,3', '2,4', "3,4"])

In [ ]:
#adata.obsm["X_pca_1_40"] = adata.obsm["X_pca"][:,1:41]

In [ ]:
sc.pp.neighbors(adata, n_neighbors=25, n_pcs=50)#, use_rep="X_pca_1_40")

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution = 0.2)

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase",'sample','Classification', 'total_counts', 'pct_counts_hMT'],ncols=2)

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='sarc')

In [ ]:
#sc.pl.embedding_density(adata, basis='umap', key='umap_density_sarc', group='wk4')

In [ ]:
sc.pl.umap(adata, color=['COL1A1','DES','MDM2',"CDK4"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['FBN1','LUM',"POSTN"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['COL4A1','PECAM1','CD34',"RGS5"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['TPM2','TOP2A','VIM',"ISG15"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['KRT18','GAPDH','LDHB',"TPI1"],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['ACTA2'], ncols=2, use_raw=True)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')#,use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon',use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
writeLoc

In [ ]:
#adata = sc.read(resultsFile)

In [ ]:
resultsFile

In [ ]:
adata.write(resultsFile)

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')

In [ ]:
ctwk1QC = writeLoc + 'sarcPDX_Treat_ctw1_UPS2236_V4_QC.h5ad'  
adataCtWk1 = sc.read(ctwk1QC)
adataCtWk1.layers["counts"] = adataCtWk1.X.copy()
adataCtWk1

In [ ]:
sc.pp.normalize_total(adataCtWk1, target_sum=1e4)
sc.pp.log1p(adataCtWk1)

In [ ]:
wk3QC = writeLoc + 'sarcPDX_Treat_wk3_UPS2236_V4_QC.h5ad'  
adataWk3 = sc.read(wk3QC)
adataWk3.layers["counts"] = adataWk3.X.copy()
adataWk3

In [ ]:
sc.pp.normalize_total(adataWk3, target_sum=1e4)
sc.pp.log1p(adataWk3)

In [ ]:
adata = adataCtWk1.concatenate(adataWk3, batch_categories=['ctwk1', 'wk3'])
adata

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"],jitter=0.4, multi_panel=True, groupby="batch")

In [ ]:
#ctl = [x in ["ctl"] for x in adata.obs["sarc"]]
#adata = adata[ctl,]
#adata

In [ ]:
#adata.var_names = [x.split("_")[1] for x in adata.var_names]
#adata.var_names_make_unique()

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')

In [ ]:
#adata.write('../../write/sarcPDX_Treat_all_V4_UPS2236_QC.h5ad')
adata = sc.read_h5ad('../../write/sarcPDX_Treat_all_V4_UPS2236_QC.h5ad')
adata

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../../data/regev_lab_cell_cycle_genes_Human.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
adata_cc_genes = adata[:, cell_cycle_genes]
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color=['phase','batch'])

In [ ]:
adata = adata[adata.obs.phase=="G1",:]

In [ ]:
adata.write('../../write/sarcPDX_Treat_G1_V4_UPS2236_QC.h5ad')

In [ ]:
#adata_cc_genes.var[adata_cc_genes.var["highly_variable"]]

In [ ]:
sc.pp.highly_variable_genes(adata, batch_key="batch")#, flavor="seurat", n_top_genes=2000)

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
minMean = 0.045
maxMean = 2.9
minDisp = 0.35

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

means = adata.var[["means"]][adata.var[["means"]] > np.exp(-14)]#adata.var[["means"]],
axs[0].hist(np.log(means), bins=100)#, log=True),
axs[0].axvline(np.log(minMean), color='k', linestyle='dashed', linewidth=1)
axs[0].axvline(np.log(maxMean), color='k', linestyle='dashed', linewidth=1)
axs[0].set_title('Gene means counts')
axs[0].set_xlabel('means')
axs[0].set_ylabel('counts')

dispNorm = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-10)]#adata.var[["means"]],
axs[1].hist(np.log(dispNorm), bins=100)#, log=True),
axs[1].axvline(np.log(minDisp), color='k', linestyle='dashed', linewidth=1)
axs[1].set_title('Gene dispersions counts')
axs[1].set_xlabel('dispersions')
axs[1].set_ylabel('counts')

sc.pp.highly_variable_genes(adata, min_disp=minDisp, min_mean=minMean, max_mean=maxMean, batch_key="batch")
print(sum(adata.var.highly_variable))

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
adata = adata[:, np.logical_and(np.logical_not(adata.var.hMT), adata.var.highly_variable)]

In [ ]:
#adata = adata.raw.to_adata()

In [ ]:
#sc.pp.regress_out(adata, ['S_score', 'G2M_score'])
#sc.pp.regress_out(adata, ['total_counts', 'pct_counts_hMT'])#])#

In [ ]:
#sc.pp.scale(adata, max_value=4)

In [ ]:
sc.tl.pca(adata, n_comps=100,svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=100, log=True)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=40, n_pcs=60)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution = 0.9)

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase","sarc",'Classification'],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['total_counts', 'pct_counts_hMT'],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['COL1A1','DES','MDM2',"VEGFA"],ncols=4)

In [ ]:
sc.pl.umap(adata, color=['FBN1','BNIP3',"PGK1","NDRG1"],ncols=4)

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')#,use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(100).to_csv("hvg/pdxTreated_UPS2236_V4.csv")

0 - FAP
1 - infG
2 - early
3 - Hypox
4 - Nfkb
5 - inflFAP
6 - sam2A

In [ ]:
features = ["FAP","infG","MusImm","Hypox","Nfkb","inflFAP","sam2A"]
sc.tl.leiden(adata, resolution=0.9, key_added = "chemoLabel")
adata.rename_categories('chemoLabel', features)

In [ ]:
sc.pl.umap(adata, color=["chemoLabel"],legend_loc="on data")

In [ ]:
sc.tl.filter_rank_genes_groups(adata, min_in_group_fraction=0.25, min_fold_change=1, max_out_group_fraction=0.5)
sc.pl.rank_genes_groups(adata, key='rank_genes_groups_filtered', sharey=False)

In [ ]:
adata

In [ ]:
adata.write('../../write/sarcPDX_Treat_all_UPS2236_V4.h5ad')

In [ ]:
sc.tl.dendrogram(adata, groupby="leiden", n_pcs=60)
sc.pl.rank_genes_groups_dotplot(adata, key='rank_genes_groups_filtered',n_genes=4)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups_filtered']['names'])

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase",'sarc','Classification'],ncols=2)

In [ ]:
sc.tl.embedding_density(adata, basis='umap', groupby='sarc')

In [ ]:
sc.pl.embedding_density(adata, basis='umap', key='umap_density_sarc')

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20)

In [ ]:
list(pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20).iloc[:,1])

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_dotplot(adata, groupby = 'leiden',n_genes=4)

In [ ]:
sc.tl.rank_genes_groups(adata, 'sarc', method='wilcoxon')#,use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
sc.pl.umap(adata, color=['leiden',"feature",'sarc','Classification'],ncols=2)

In [ ]:
#treatHvg = pd.read_csv("hvg/treatedHvg.csv")
#treatHvg

In [ ]:
#sc.tl.score_genes(adata, treatHvg["3"],score_name='treated_3')
#sc.tl.score_genes(adata, treatHvg["7"],score_name='treated_7')
#sc.pl.umap(adata,color=["treated_3","treated_7","leiden"])

In [ ]:
adata.write('../../write/sarcPDX_Treat_all_UPS2236_V4.h5ad')

In [ ]:
adata = sc.read('../../write/sarcPDX_Treat_all_UPS2236_V4.h5ad')
adata

In [ ]:
adataP = sc.read('../../write/scSarc3_PDX_V3_QC.h5ad')
adataP

In [ ]:
adataT = sc.read('../../write/sarcPDX_Treat_all_UPS2236_QC.h5ad')
adataT = adataT[adataT.obs.sarc=="ctl"]

In [ ]:
adata = sc.concat({"treat":adataT, "orig":adataP}, label="run")
adata

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', "pct_counts_hMT"],jitter=0.4, multi_panel=True, groupby="run")

In [ ]:
#ctl = [x in ["ctl"] for x in adata.obs["sarc"]]
#adata = adata[ctl,]
#adata

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata)

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../../data/regev_lab_cell_cycle_genes_Human.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
adata_cc_genes = adata[:, cell_cycle_genes]
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color='phase')

In [ ]:
#adata_cc_genes.var[adata_cc_genes.var["highly_variable"]]

In [ ]:
sc.pp.highly_variable_genes(adata, batch_key="run")#, flavor="seurat", n_top_genes=2000)

In [ ]:
adata.var

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
x = adata.var[["means"]][adata.var[["means"]] > np.exp(-10)]#adata.var[["means"]]
plt.hist(np.log(x), bins=100)#, log=True)
plt.axvline(np.log(0.05), color='k', linestyle='dashed', linewidth=1)
plt.axvline(np.log(3), color='k', linestyle='dashed', linewidth=1)
plt.title('Gene means counts')
plt.xlabel('means')
plt.ylabel('counts')
plt.show()

In [ ]:
x = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-4)]#adata.var[["dispersions"]]
plt.hist(np.log(x), bins=50)#, log=True)
plt.axvline(np.log(0.5), color='k', linestyle='dashed', linewidth=1)
plt.title('Gene dispersions counts')
plt.xlabel('dispersions')
plt.ylabel('counts')
plt.show()

In [ ]:
sc.pp.highly_variable_genes(adata, min_disp=0.5, min_mean=0.05, max_mean=3,batch_key="run")

In [ ]:
adata.var

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.raw = adata

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
adata = adata[:, adata.var.highly_variable]

In [ ]:
adata.var

In [ ]:
#sc.pp.regress_out(adata, ['S_score', 'G2M_score'])
sc.pp.regress_out(adata, ['S_score', 'G2M_score','total_counts', 'pct_counts_hMT'])#])#

In [ ]:
sc.pp.scale(adata, max_value=4)

In [ ]:
sc.tl.pca(adata, n_comps=100,svd_solver='arpack')

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs=100, log=True)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=60)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution = 0.6)

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase",'run','Classification'],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase",'sarc','Classification','total_counts', 'pct_counts_hMT'],ncols=2)

In [ ]:
sc.pl.umap(adata, color=['leiden',"phase",'Classification'],ncols=2)